In [34]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import *

In [35]:
def get_financial_report(ticker):
#build URLs
    urlfinancials = 'https://www.marketwatch.com/investing/stock/'+ticker+'/financials'
    urlbalancesheet = 'https://www.marketwatch.com/investing/stock/'+ticker+'/financials/balance-sheet'
#request the data using beautiful soup
    text_soup_financials = BeautifulSoup(requests.get(urlfinancials).text,"html") 
    text_soup_balancesheet = BeautifulSoup(requests.get(urlbalancesheet).text,"html")
    # build lists for Income statement
    titlesfinancials = text_soup_financials.findAll('td', {'class': 'rowTitle'})
    epslist=[]
    netincomelist = []
    longtermdebtlist = [] 
    interestexpenselist = []
    ebitdalist= []
    #load data into lists if the row title is found
    for title in titlesfinancials:
        if 'EPS (Basic)' in title.text:
            epslist.append ([td.text for td in  title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'Net Income' in title.text:
            netincomelist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'Interest Expense' in title.text:
            interestexpenselist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'EBITDA' in title.text:
            ebitdalist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
    # find the table headers for the Balance sheet
    titlesbalancesheet = text_soup_balancesheet.findAll('td', {'class': 'rowTitle'})
    equitylist=[]
    for title in titlesbalancesheet:
        if 'Total Shareholders\' Equity' in title.text:
            equitylist.append( [td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'Long-Term Debt' in title.text:
            longtermdebtlist.append( [td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
    #get the data from the income statement lists 
    #use helper function get_element
    eps = get_element(epslist,0)
    epsGrowth = get_element(epslist,1)
    netIncome = get_element(netincomelist,0)
    shareholderEquity = get_element(equitylist,0)
    roa = get_element(equitylist,1)
    longtermDebt = get_element(longtermdebtlist,0)
    interestExpense =  get_element(interestexpenselist,0)
    ebitda = get_element(ebitdalist,0)

    # load all the data into dataframe 
    fin_df= pd.DataFrame({'eps': eps,'eps Growth': epsGrowth,'net Income': netIncome,'shareholder Equity': shareholderEquity,'roa': roa,'longterm Debt': longtermDebt,'interest Expense': interestExpense,'ebitda': ebitda}, index=range(date.today().year-5,date.today().year))

    fin_df.reset_index(inplace=True)

    return fin_df


In [36]:
                         
#helper function
def get_element(list,element):
    try:
        return list[element]
    except:
        return '-'
            

In [37]:
get_financial_report('TSLA')

,index,eps,eps Growth,net Income,shareholder Equity,roa,longterm Debt,interest Expense,ebitda
0,2015,(1.39),-,(888.66M),1.08B,13.43%,2.27B,118.85M,(294.04M)
1,2016,(0.94),32.47%,(773.05M),4.75B,20.97%,7.39B,198.81M,256.56M
2,2017,(2.37),-152.78%,(2.24B),4.24B,14.79%,11.15B,471.26M,(1.68M)
3,2018,(1.14),51.61%,(1.06B),4.92B,16.55%,11.12B,663.07M,1.65B
4,2019,(0.97),14.92%,(775M),6.62B,19.29%,12.63B,685M,2.23B
